# Working With STAC

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/main?filepath=docs%2Fexamples%2F%2Fnotebooks%2FWorking_with_STAC_simple.ipynb)

### STAC: SpatioTemporal Asset Catalog

> The SpatioTemporal Asset Catalog (STAC) specification aims to standardize the way geospatial assets are exposed online and queried. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time. The initial focus is primarily remotely-sensed imagery (from satellites, but also planes, drones, balloons, etc), but the core is designed to be extensible to SAR, full motion video, point clouds, hyperspectral, LiDAR and derived data like NDVI, Digital Elevation Models, mosaics, etc.

Ref: https://github.com/radiantearth/stac-spechttps://github.com/radiantearth/stac-spec

Using STAC makes data indexation and discovery really easy. In addition to the Collection/Item/Asset (data) specifications, data providers are also encouraged to follow a STAC API specification:  https://github.com/radiantearth/stac-api-spec

> The API is compliant with the OGC API - Features standard (formerly known as OGC Web Feature Service 3), in that it defines many of the endpoints that STAC uses. A STAC API should be compatible and usable with any OGC API - Features clients. The STAC API can be thought of as a specialized Features API to search STAC Catalogs, where the features returned are STAC Items, that have common properties, links to their assets and geometries that represent the footprints of the geospatial assets.


## Requirements

To be able to run this notebook you'll need the following requirements:
- folium
- httpx
- rasterio

`!pip install folium httpx rasterio`

In [1]:
# Uncomment this line if you need to install the dependencies
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import httpx

from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.
stac_item = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_30TVT_20221112_0_L2A"

In [4]:
item = httpx.get(stac_item).json()
print(item)

{'type': 'Feature', 'stac_version': '1.0.0', 'id': 'S2A_30TVT_20221112_0_L2A', 'properties': {'created': '2022-11-14T06:54:49.284Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 0.005979, 'proj:epsg': 32630, 'mgrs:utm_zone': 30, 'mgrs:latitude_band': 'T', 'mgrs:grid_square': 'VT', 'grid:code': 'MGRS-30TVT', 'view:sun_azimuth': 169.467826196677, 'view:sun_elevation': 24.259740600657594, 's2:degraded_msi_data_percentage': 0, 's2:nodata_pixel_percentage': 0.000226, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0, 's2:cloud_shadow_percentage': 0.002296, 's2:vegetation_percentage': 10.348745, 's2:not_vegetated_percentage': 2.478484, 's2:water_percentage': 87.111628, 's2:unclassified_percentage': 0.002548, 's2:medium_proba_clouds_percentage': 0.003716, 's2:high_proba_clouds_percentage': 0.000508, 's2:thin_cirrus_percentage': 0.001755, 's2:snow_ice_percentage': 0.050325, 's2:product_type': 'S2MSI2A', 's2:pro

In [5]:
for it, asset in item["assets"].items():
    print("Name:", it, "| Format:", asset["type"])

Name: aot | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: blue | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: coastal | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: granule_metadata | Format: application/xml
Name: green | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: nir | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: nir08 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: nir09 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: red | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: rededge1 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: rededge2 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: rededge3 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: scl | Format: image/tiff; application=geotiff; profile=cloud-optimized
N

In [6]:
bounds = featureBounds(item)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

In [7]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/info",
    params = (
        ("url", stac_item),
        # Get info for multiple assets
        ("assets","visual"), ("assets","red"), ("assets","blue"), ("assets","green"),
    )
).json()
print(r)

{'visual': {'bounds': [-4.337134709547373, 46.85817595750231, -2.869529638083867, 47.85370180403547], 'minzoom': 7, 'maxzoom': 13, 'band_metadata': [['b1', {}], ['b2', {}], ['b3', {}]], 'band_descriptions': [['b1', ''], ['b2', ''], ['b3', '']], 'dtype': 'uint8', 'nodata_type': 'Nodata', 'colorinterp': ['red', 'green', 'blue'], 'scales': [1.0, 1.0, 1.0], 'offsets': [0.0, 0.0, 0.0], 'driver': 'GTiff', 'count': 3, 'width': 10980, 'height': 10980, 'overviews': [2, 4, 8, 16], 'nodata_value': 0.0}, 'red': {'bounds': [-4.337134709547373, 46.85817595750231, -2.869529638083867, 47.85370180403547], 'minzoom': 7, 'maxzoom': 13, 'band_metadata': [['b1', {}]], 'band_descriptions': [['b1', '']], 'dtype': 'uint16', 'nodata_type': 'Nodata', 'colorinterp': ['gray'], 'scales': [1.0], 'offsets': [0.0], 'driver': 'GTiff', 'count': 1, 'width': 10980, 'height': 10980, 'overviews': [2, 4, 8, 16], 'nodata_value': 0.0}, 'blue': {'bounds': [-4.337134709547373, 46.85817595750231, -2.869529638083867, 47.853701804

### Display one asset

In [8]:
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "visual",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

### Select Indexes for assets

In [9]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "visual",
        "asset_bidx": "visual|3,1,2",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

In [10]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = (
        ("url", stac_item),
        ("assets", "red"),
        ("assets", "green"),
        ("assets", "blue"),
        # Most of the Sentinel L2A Assets have only one band
        # So we don't have to pass the bidx
        # ("assets_bidx", "red|1"),
        # ("assets_bidx", "green|1"),
        # ("assets_bidx", "blue|"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("rescale", "0,2000"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=11
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

Use an expression to calculate a band index (NDVI) based on information contained in multiple assets.

In [11]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = (
        ("url", stac_item),
        ("expression", "(nir-red)/(nir+red)"),  # NDVI
        # We need to tell rio-tiler that each asset is a Band
        # (so it will select the first band within each asset automatically)
        ("asset_as_band", True),
        ("rescale", "-1,1"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("colormap_name", "viridis"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

If you don't use the  `asset_as_band=True` option, you need to append the band to the asset name within the expression. For example, `nir` becomes `nir_b1`.

In [12]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = (
        ("url", stac_item),
        ("expression", "(nir_b1-red_b1)/(nir_b1+red_b1)"),  # NDVI
        ("rescale", "-1,1"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("colormap_name", "viridis"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m